## Задание 1

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Get the same results each time
np.random.seed(0)

# Load the training data
data = pd.read_csv("content/data.csv")
comments = data["comment_text"]
target = (data["target"] > 0.7).astype(int)

# Разделение данных на train и test
X_train, X_test, y_train, y_test = train_test_split(comments, target, test_size=0.3, random_state=42)

## Задание 2

In [4]:
# Инициализация CountVectorizer
vectorizer = CountVectorizer()

# Преобразование текста в числовые данные
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

## Задание 3

In [5]:
# Инициализация логистической регрессии
classifier = LogisticRegression(max_iter=2000)

# Обучение модели
classifier.fit(X_train_counts, y_train)

# Предсказание на тестовых данных
y_pred = classifier.predict(X_test_counts)

# Расчет accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9277254226100986


## Задание 4

In [7]:
def predict_toxicity(comment):
    # Преобразование комментария в числовое представление
    comment_counts = vectorizer.transform([comment])
    # Предсказание вероятности
    probability = classifier.predict_proba(comment_counts)[:, 1]
    return probability[0]


## Задание 5

In [10]:
# Примеры предсказаний
print(predict_toxicity("Apples are stupid"))  # Ожидаем значение близкое к 1
print(predict_toxicity("I love apples"))      # Ожидаем значение близкое к 0

0.9991454496873886
0.05885369712646557


## Задание 6

In [11]:
# Получаем список слов и их коэффициентов
feature_names = np.array(vectorizer.get_feature_names_out())
coefs = classifier.coef_[0]

# Индексы самых больших коэффициентов
top_indices = coefs.argsort()[-10:][::-1]

# Самые токсичные слова и их коэффициенты
toxic_words = feature_names[top_indices]
toxic_coefs = coefs[top_indices]

for word, coef in zip(toxic_words, toxic_coefs):
    print(f"Word: {word}, Coefficient: {coef}")

Word: stupid, Coefficient: 9.248914153050393
Word: idiot, Coefficient: 8.713474437073495
Word: idiots, Coefficient: 8.440952076056677
Word: stupidity, Coefficient: 7.532447834250859
Word: idiotic, Coefficient: 6.828585093096055
Word: crap, Coefficient: 6.56792271566527
Word: dumb, Coefficient: 6.450247704002257
Word: pathetic, Coefficient: 6.410854251847928
Word: hypocrite, Coefficient: 6.38542948060996
Word: moron, Coefficient: 6.351060787642749


## Задание 7

Удивительные слова: нет
Слова, которые, возможно, не должны быть в списке: "pathetic"

## Задание 8

In [12]:
# Примеры предсказаний
print(predict_toxicity("I have a christian friend"))
print(predict_toxicity("I have a muslim friend"))
print(predict_toxicity("I have a white friend"))
print(predict_toxicity("I have a black friend"))

0.18760590388569065
0.5108092195475886
0.40150015555040297
0.5887745076141645


### Анализ результатов:

1. **"I have a christian friend"**:
   - Токсичность: **0.187**
   - Комментарий кажется нейтральным и не содержит явных оскорблений или дискриминации. Модель правильно классифицирует его как относительно безопасный.

2. **"I have a muslim friend"**:
   - Токсичность: **0.511**
   - Этот комментарий также нейтрален, но модель выдает более высокую вероятность токсичности. Это может указывать на потенциальную предвзятость модели в отношении религии, особенно мусульманства.

3. **"I have a white friend"**:
   - Токсичность: **0.402**
   - Комментарий нейтрален, но модель снова выдает среднюю вероятность токсичности. Это может быть связано с тем, что упоминание расы само по себе воспринимается моделью как потенциально проблематичное.

4. **"I have a black friend"**:
   - Токсичность: **0.589**
   - Здесь модель выдает самую высокую вероятность токсичности среди всех комментариев. Это вызывает серьезные вопросы об этичности модели, так как комментарий остается нейтральным и не содержит явных оскорблений.

---

### Выводы:

1. **Предвзятость (bias)**:
   - Модель демонстрирует различную чувствительность к упоминанию различных групп. Особенно примечательно, что комментарии, связанные с мусульманами и чернокожими людьми, получают значительно более высокие значения токсичности, чем комментарии о христианах или белых людях.
   - Это может указывать на то, что модель обучена на данных, где такие упоминания часто ассоциировались с токсичными комментариями, что привело к формированию предвзятости.

2. **Этика**:
   - Высокая токсичность для комментариев, содержащих упоминания определенных религий или рас, является этически проблематичной. Такое поведение модели может способствовать дискриминации и стереотипам, даже если комментарии сами по себе нейтральны.
   - Это подчеркивает необходимость дальнейшего анализа и корректировки модели, чтобы избежать дискриминационного воздействия.

---

### Итоговый ответ:
- **Есть ли bias?** Да, модель показывает различную чувствительность к упоминанию религии и расы, что указывает на наличие предвзятости.
- **Этичен ли он?** Нет, текущее поведение модели неэтично, так как оно может способствовать дискриминации и стереотипам. Необходимо провести дополнительную работу по устранению этих проблем.

## Задание 9

Модель демонстрирует предвзятость к комментариям о мусульманах, что может нарушить все четыре критерия этики ИИ:
1. **Демографический паритет**: Меньшая доля комментариев о мусульманах будет отмечена как нетоксичные.
2. **Равные возможности**: Высокий истинно положительный показатель (TPR) для комментариев о мусульманах.
3. **Одинаковая точность**: Разная точность модели для разных групп.
4. **Группа по незнанию**: Удаление информации о религии может не полностью избавить модель от предвзятости из-за прокси-переменных.

## Задание 10

1. Использование более сложных моделей машинного обучения, таких как случайные леса или нейронные сети.

2. Удаление или снижение веса слов, связанных с определенными группами или религиями, чтобы снизить предвзятость модели.

## Задание 11

1. Использование методов фильтрации для удаления явно токсичных слов или фраз.

2. Признаки, учитывающие контекст и эмоциональный оттенок комментариев, а не только наличие определенных слов.